In [ ]:
import torch

# Example rollout data
factor_values = {
    "x": [0.5, 0.7, 0.4, 0.8],  # x position
    "y": [0.2, 0.3, 0.1, 0.4],  # y position
    }
outcomes = [1, 0, 0, 1]  # success = 1
# Concatenate factor values and outcomes into training data

train_X = torch.tensor([i for i in factor_values.values()]).T.to(torch.double)
train_Y = torch.tensor(outcomes).unsqueeze(-1).to(torch.double)

In [ ]:
# Fit a Gaussian Process model to data (https://botorch.org/docs/models)
# SingleTaskGP: a single-task exact GP that supports both inferred and observed noise. When noise observations are not provided, it infers a homoskedastic noise level.
# MixedSingleTaskGP: a single-task exact GP that supports mixed search spaces, which combine discrete and continuous features.
# SaasFullyBayesianSingleTaskGP: a fully Bayesian single-task GP with the SAAS prior. This model is suitable for sample-efficient high-dimensional Bayesian optimization.
# SingleTaskVariationalGP: an approximate model for faster computation when you have a lot of data or your responses are non-Gaussian.

from botorch.models import SingleTaskGP
from botorch.models.transforms import Normalize
from botorch.fit import fit_gpytorch_mll  # fit surrogate model (suitable for exact GPs)
from gpytorch.mlls import ExactMarginalLogLikelihood

gp = SingleTaskGP(
    train_X=train_X,
    train_Y=train_Y,
    input_transform=Normalize(d=2),
)

mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_mll(mll)

ExactMarginalLogLikelihood(
  (likelihood): GaussianLikelihood(
    (noise_covar): HomoskedasticNoise(
      (noise_prior): LogNormalPrior()
      (raw_noise_constraint): GreaterThan(1.000E-04)
    )
  )
  (model): SingleTaskGP(
    (likelihood): GaussianLikelihood(
      (noise_covar): HomoskedasticNoise(
        (noise_prior): LogNormalPrior()
        (raw_noise_constraint): GreaterThan(1.000E-04)
      )
    )
    (mean_module): ConstantMean()
    (covar_module): RBFKernel(
      (lengthscale_prior): LogNormalPrior()
      (raw_lengthscale_constraint): GreaterThan(2.500E-02)
    )
    (outcome_transform): Standardize()
    (input_transform): Normalize()
  )
)

In [21]:
# Construct an acquisition function (https://botorch.org/docs/acquisition)
# Joint Entropy Search (JES)
# Bayesian Active Learning By Disagreement (BALD)
# GIBBON (General-purpose Information-Based Bayesian OptimisatioN)

from botorch.acquisition.utils import get_optimal_samples

tkwargs = {"dtype": torch.double, "device": "cpu"}
num_samples = 12

# Bounds of the search space. If the model inputs are normalized, the bounds should be normalized as well.
bounds = torch.tensor([[-3.0, -3.0], [3.0, 3.0]], **tkwargs)  # 3 std devs

optimal_inputs, optimal_outputs = get_optimal_samples(
    gp, bounds=bounds, num_optima=num_samples
)

from botorch.acquisition.joint_entropy_search import qJointEntropySearch

jes_lb = qJointEntropySearch(
    model=gp,
    optimal_inputs=optimal_inputs,
    optimal_outputs=optimal_outputs,
    estimation_type="LB",
)

In [23]:
# Optimize the acquisition function
from botorch.optim import optimize_acqf

candidate, acq_value = optimize_acqf(
    acq_function=jes_lb,
    bounds=bounds,
    q=1,
    num_restarts=4,
    raw_samples=256,
)
print("JES-LB: candidate={}, acq_value={}".format(candidate, acq_value))

JES-LB: candidate=tensor([[0.9966, 2.1044]], dtype=torch.float64), acq_value=0.20906776660708734
